In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

## Funzioni di trasferimento

In [ ]:
def Mod_Extract(f):
    r = f()
    return r[0]

def RC_R(W, R, C):
    mod = R/(np.sqrt(R**2 + 1/(W*C)**2))
    fase = np.arctan(1/(W*C*R))

    return mod, fase

def RC_C(W, R, C):
    mod = 1/np.sqrt(1+(R*C*W)**2)
    fase = np.arctan(R*C*W)

    return mod, fase

def RL_R(W, R, L):
    mod = R/(R**2 + (W*L)**2)
    fase = np.arctan(W*L/R)

    return mod, fase

def RL_L(W, R, L):
    mod = W*L/(R**2 + (W*L)**2)
    fase = np.arctan(W*L/R)

def RLC_R(W, R, C, L):
    mod = R/(np.sqrt((R**2)+((W*L)-(1/(W*C))**2)))
    fase = np.arctan((((W**2)*L*C)-1)/(W*C*R))
    
    return mod, fase

def RLC_C(W, R, C, L):
    mod =  1/(np.sqrt((1-((W**2)*L*C))**2)+((W**2)*(C**2)*(R**2)))
    fase = np.arctan((W*C*R)/(1-((W**2)*L*C)))
    
    return mod, fase


def RLC_L(W, R, C, L):
    mod = ((W**2)*(L**2))/(np.sqrt((R**2) + ((W*L)-(1/(W*C)))**2))
    fase = np.arctan((((W**2)*L*C)-1)/(W*C*R))

    return mod, fase

# Prima parte RC RL 

# CIRCUITO RC
 Capacità utilizzata= $99 \pm 1$ nF \
 Resistenza= $100\pm 1 \Omega$ (check)
 
 Incertezza fase (CH2-CH1) = $\pm 1$ \
 Incertezza Ampiezza V_in(CH1) = $\pm 0.05$   Trasportarla su asse x e asse y \
 Incertezza Ampiezza V_out (CH2) = $\pm 0.05$ \
 Incertezza differenza (MATH)  = $\pm 0.01$ \

 le fasi nel foglio sono scritte in gradi

In [ ]:
dati1 = pd.read_excel("dati_rc.xlsx")

RC_freq = dati1["Frequenza"]
RC_amp_A = dati1["Ampiezza Va"]
RC_amp_B = dati1["Ampiezza Vb"]
RC_amp_ab = dati1["Ampiezza Va-b"]
RC_fase_A = dati1["ddf a/b"] 
RC_fase_AB = dati1["ddf a-b/a"]

In [ ]:
# PLot dati RC resitenza

RC_H_R = [i/j for i,j in zip(RC_amp_B, RC_amp_A)]

plt.plot(RC_freq,RC_H_R, marker = 'o')
plt.xscale('log')
plt.grid()
plt.show()

In [ ]:
# fit dati resistenza

Res = 100 #ohm
Cap = 100e-9 #farad

# pars1, vals1, ers1, *other1 = LS_fit(RC_freq, RC_H_R, RC_R[0], R = Res, C = Cap)
# separare le funzioni sopra

In [ ]:
x = np.linspace(1, 3000000, 1000)
y = -RC_R(x, Res, Cap)[1]

plt.scatter(RC_freq, [np.deg2rad(i) for i in RC_fase_AB])
plt.plot(x, y)
plt.grid()
plt.show()

In [ ]:
#plot dati capacità

RC_H_C = [i/j for i,j in zip(RC_amp_ab, RC_amp_A)]

plt.plot(RC_freq, RC_H_C, marker='o')
plt.xscale('log')
plt.grid()
plt.show()

# CIRCUITO RL


Induttanza utilizzata =   \
Resistenza utilizzata =   \
Incertezza fase =   \
Incertezza ampiezza V_in =  \ 
Incertezza ampiezza V_out =  \
Incertezza differenza (MATH) =   


# CIRCUITO RLC

Resistenza utilizzata = 
Induttanza utilizzata = 
Capacità utilizzata = 

